In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
#just for the notebook
%run implementations
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
DATA_TEST_PATH = '../data/test.csv'
y, tX, train_ids = load_csv_data(DATA_TRAIN_PATH)
subsmission_y, submission_x, test_ids = load_csv_data(DATA_TEST_PATH)

In [3]:
tX.shape

(250000, 30)

In [4]:
DEGREE = 3

In [5]:
x_train, y_train, x_test, y_test = split_data(tX, y, 0.8, seed=1)

In [6]:
# Append y values as column to later divide y into buckets corresponding with x values
x_train = np.column_stack((x_train, y_train))
x_test = np.column_stack((x_test, y_test))

In [7]:
# get all the needed buckets
buckets = get_buckets(x_train)
# y
y_buckets = []
for i in range(len(buckets)):
    y_buckets.append(buckets[i][:, -1])
    buckets[i] = np.delete(buckets[i], -1, 1)
# test
test_buckets = get_buckets(x_test)
submission_x_buckets = get_buckets(submission_x.copy())
# test y
test_y_buckets = []
for i in range(len(test_buckets)):
    test_y_buckets.append(test_buckets[i][:, -1])
    test_buckets[i] = np.delete(test_buckets[i], -1, 1)

In [8]:
# Normalize the matrices
for i in range(len(buckets)):
    buckets[i] = normalize(buckets[i])
    test_buckets[i] = normalize(test_buckets[i])
    submission_x_buckets[i] = normalize(submission_x_buckets[i])

In [11]:
# Add Intercept
for i in range(len(buckets)):
    buckets[i] = np.column_stack((np.ones((buckets[i].shape[0], 1)), buckets[i]))
    test_buckets[i] = np.column_stack((np.ones((test_buckets[i].shape[0], 1)), test_buckets[i]))
    submission_x_buckets[i] = np.column_stack((np.ones((submission_x_buckets[i].shape[0], 1)),
                                              submission_x_buckets[i]))

In [17]:
weights = []
for i in range(len(buckets)):
    w_rr, loss_rr = ridge_regression(y=y_buckets[i], tx=build_poly(buckets[i], 2), lambda_=0.0001)
    weights.append(w_rr)

In [13]:
def accuracy(train_buckets, test_buckets, test_y_buckets, weights, degree, lambda_):
    correct_predictions = 0
    len_data = 0
    for i in range(len(train_buckets)):
        rr_accuracy = compute_accuracy(weights[i], test_buckets[i], test_y_buckets[i])
        correct_predictions += (rr_accuracy * len(test_buckets[i]))
        len_data += len(test_buckets[i])
    total_accuracy = correct_predictions / len_data
    print(f"Accuracy = {total_accuracy} with degree = {degree} and lambda = {lambda_}")
    return total_accuracy

In [ ]:
# Compute polynomial
def compute_poly():
    for i in range(len(buckets))
    test_buck

In [18]:
acc = accuracy(train_buckets=buckets, test_buckets=test_buckets, test_y_buckets=test_y_buckets,
                            weights=weights, degree=0, lambda_=0.05)

ValueError: shapes (14738,31) and (66,) not aligned: 31 (dim 1) != 66 (dim 0)

In [ ]:
for i in range(len(buckets)):
    best_degree_selection(y=y_buckets[i], x=buckets[i], degrees=range(7, 12), 
                      k_fold=5, lambdas=np.linspace(0.1, 1, 10))

best lambdas : 0.30000000000000004
best degree 7
best lambdas : 0.1
best degree 7


In [ ]:
type(y_buckets)

## Do your thing crazy machine learning thing here :) ...

## Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
OUTPUT_PATH = '../data/submission.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)